In [2]:
"""
%conda install -c conda-forge langchain
%conda install -c conda-forge pypdf

%pip install langchain_ollama
%pip install langchain_community
"""

'\n%conda install -c conda-forge langchain\n%conda install -c conda-forge pypdf\n\n%pip install langchain_ollama\n%pip install langchain_community\n'

In [3]:
"""
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
"""

'\nimport getpass\nimport os\n\nos.environ["LANGCHAIN_TRACING_V2"] = "true"\nos.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")\n'

In [4]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="llama3.2",
    temperature=0,
)

In [5]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "example_data/nke-10k-2023.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

107


In [6]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
F

{'source': 'example_data/nke-10k-2023.pdf', 'page': 0}


## Splitting

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

516

## Embeddings

In [8]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="nomic-embed-text"
)

In [9]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 768

[-0.022926938, 0.07179706, -0.19206722, -0.06737124, 0.026634539, -0.024445355, 0.052351013, 0.010194804, 0.075033516, 0.008327751]


## Vector stores

In [15]:
from langchain_chroma import Chroma

vector_store = Chroma(embedding_function=embeddings)
ids = vector_store.add_documents(documents=all_splits)

Usage

In [16]:
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)

print(results[0])

page_content='direct to consumer operations sell products through the following number of retail stores in the United States:
U.S. RETAIL STORES NUMBER
NIKE Brand factory stores 213 
NIKE Brand in-line stores (including employee-only stores) 74 
Converse stores (including factory stores) 82 
TOTAL 369 
In the United States, NIKE has eight significant distribution centers. Refer to Item 2. Properties for further information.
2023 FORM 10-K 2' metadata={'page': 4, 'source': 'example_data/nke-10k-2023.pdf', 'start_index': 3125}


### Async query

In [17]:
results = await vector_store.asimilarity_search("When was Nike incorporated?")

print(results[0])

page_content='transition of NIKE Brand businesses in certain countries within APLA to third-party distributors.
The Company's NIKE Direct operations are managed within each NIKE Brand geographic operating segment. Converse is also a reportable segment for the Company
and operates in one industry: the design, marketing, licensing and selling of athletic lifestyle sneakers, apparel and accessories.
Global Brand Divisions is included within the NIKE Brand for presentation purposes to align with the way management views the Company. Global Brand Divisions
revenues include NIKE Brand licensing and other miscellaneous revenues that are not part of a geographic operating segment. Global Brand Divisions costs represent
demand creation and operating overhead expense that include product creation and design expenses centrally managed for the NIKE Brand, as well as costs associated
with NIKE Direct global digital operations and enterprise technology.
(1)
2023 FORM 10-K 84' metadata={'page': 86, '

### Return scores

In [18]:
# Note that providers implement different scores; the score here
# is a distance metric that varies inversely with similarity.

results = vector_store.similarity_search_with_score("What was Nike's revenue in 2023?")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.31971192359924316

page_content='Table of Contents
FISCAL 2023 NIKE BRAND REVENUE HIGHLIGHTSThe following tables present NIKE Brand revenues disaggregated by reportable operating segment, distribution channel and major product line:
FISCAL 2023 COMPARED TO FISCAL 2022
• NIKE, Inc. Revenues were $51.2 billion in fiscal 2023, which increased 10% and 16% compared to fiscal 2022 on a reported and currency-neutral basis, respectively.
The increase was due to higher revenues in North America, Europe, Middle East & Africa ("EMEA"), APLA and Greater China, which contributed approximately 7, 6,
2 and 1 percentage points to NIKE, Inc. Revenues, respectively.
• NIKE Brand revenues, which represented over 90% of NIKE, Inc. Revenues, increased 10% and 16% on a reported and currency-neutral basis, respectively. This
increase was primarily due to higher revenues in Men's, the Jordan Brand, Women's and Kids' which grew 17%, 35%,11% and 10%, respectively, on a wholesale
equivalent basis.' meta

Return documents based on similarity to an embedded query

In [19]:
embedding = embeddings.embed_query("How were Nike's margins impacted in 2023?")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

page_content='Enterprise Resource Planning Platform, data and analytics, demand sensing, insight gathering, and other areas to create an end-to-end technology foundation, which we
believe will further accelerate our digital transformation. We believe this unified approach will accelerate growth and unlock more efficiency for our business, while driving
speed and responsiveness as we serve consumers globally.
FINANCIAL HIGHLIGHTS
• In fiscal 2023, NIKE, Inc. achieved record Revenues of $51.2 billion, which increased 10% and 16% on a reported and currency-neutral basis, respectively
• NIKE Direct revenues grew 14% from $18.7 billion in fiscal 2022 to $21.3 billion in fiscal 2023, and represented approximately 44% of total NIKE Brand revenues for
fiscal 2023
• Gross margin for the fiscal year decreased 250 basis points to 43.5% primarily driven by higher product costs, higher markdowns and unfavorable changes in foreign
currency exchange rates, partially offset by strategic pricing action

## Retrievers


In [20]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)


retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(metadata={'page': 4, 'source': 'example_data/nke-10k-2023.pdf', 'start_index': 3125}, page_content='direct to consumer operations sell products through the following number of retail stores in the United States:\nU.S. RETAIL STORES NUMBER\nNIKE Brand factory stores 213 \nNIKE Brand in-line stores (including employee-only stores) 74 \nConverse stores (including factory stores) 82 \nTOTAL 369 \nIn the United States, NIKE has eight significant distribution centers. Refer to Item 2. Properties for further information.\n2023 FORM 10-K 2')],
 [Document(metadata={'page': 86, 'source': 'example_data/nke-10k-2023.pdf', 'start_index': 3033}, page_content="transition of NIKE Brand businesses in certain countries within APLA to third-party distributors.\nThe Company's NIKE Direct operations are managed within each NIKE Brand geographic operating segment. Converse is also a reportable segment for the Company\nand operates in one industry: the design, marketing, licensing and selling of a

In [21]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

[[Document(metadata={'page': 4, 'source': 'example_data/nke-10k-2023.pdf', 'start_index': 3125}, page_content='direct to consumer operations sell products through the following number of retail stores in the United States:\nU.S. RETAIL STORES NUMBER\nNIKE Brand factory stores 213 \nNIKE Brand in-line stores (including employee-only stores) 74 \nConverse stores (including factory stores) 82 \nTOTAL 369 \nIn the United States, NIKE has eight significant distribution centers. Refer to Item 2. Properties for further information.\n2023 FORM 10-K 2')],
 [Document(metadata={'page': 86, 'source': 'example_data/nke-10k-2023.pdf', 'start_index': 3033}, page_content="transition of NIKE Brand businesses in certain countries within APLA to third-party distributors.\nThe Company's NIKE Direct operations are managed within each NIKE Brand geographic operating segment. Converse is also a reportable segment for the Company\nand operates in one industry: the design, marketing, licensing and selling of a